### This notebook acts like an issuer in blockchain. What we do is to generate a patient did and write them to buildernet. Furthermore, it can also issue personal data to patients' wallet to protect their private data from hackers.


### 1. Initialise a controller for Hospital Agent

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8072
WEBHOOK_BASE = ""
ADMIN_URL = "http://hospital-agent:8071"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
# API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)

In [3]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


### 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [4]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)

Subscribing too: issue_credential
Subscribing too: connections


### 3. Check Connection

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'state': 'active', 'routing_state': 'none', 'invitation_mode': 'once', 'their_role': 'inviter', 'their_did': '4gqtJKna9qZYztcAGkUKDr', 'request_id': 'c3a5e87d-47b6-48b3-8c98-ff2bdb2c7570', 'my_did': 'UFHBQiVeARLVd576EuePnN', 'created_at': '2023-01-06 04:54:24.471053Z', 'rfc23_state': 'completed', 'invitation_key': '5sUS3jiuXTW96bs68X4iBzPeHGSrymakUFtTfCKFSafQ', 'accept': 'manual', 'updated_at': '2023-01-06 04:54:44.732187Z', 'connection_id': '6cab2195-9989-40a8-9564-7130582f4c8d', 'their_label': 'THE_WALLET'}]
Connection : {'state': 'active', 'routing_state': 'none', 'invitation_mode': 'once', 'their_role': 'inviter', 'their_did': '4gqtJKna9qZYztcAGkUKDr', 'request_id': 'c3a5e87d-47b6-48b3-8c98-ff2bdb2c7570', 'my_did': 'UFHBQiVeARLVd576EuePnN', 'created_at': '2023-01-06 04:54:24.471053Z', 'rfc23_state': 'completed', 'invitation_key': '5sUS3jiuXTW96bs68X4iBzPeHGSrymakUFtTfCKFSafQ', 'accept': 'manual', 'updated_at': '2023-01-06 04:54:44.732187Z', 'connection_id': '6cab2195-9

### 3b. (OPTIONAL) Write a Schema to the Ledger

For more details see the schema-api notebook

Note: You will only be able to do this once unless you change the schema_name or version. Once you have a schema written to the ledger it can be reused by multiple entities

In [6]:
# Define you schema name - must be unique on the ledger
schema_name = "MedicalRecords"
# Can version the schema if you wish to update it
schema_version = "0.0.3"
# Define any list of attributes you wish to include in your schema
attributes = ["fullname", "age", "date", "doctorname", "complaint", "examination results", "final treatment", "medicine", "nextappointment"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PEcmuSQ7G3z771J7dse1R5:2:MedicalRecords:0.0.3


### 4. Write a Credential Definition to the Ledger


Note: Again this can only be done once per issuer, per schema_id.



In [7]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)


PEcmuSQ7G3z771J7dse1R5:3:CL:85:default


### 5. Populate the Attribues to Issue to Idenity Holder (User)


We will issue a credential to the identity holder consisting of the following attributes:

credential_attributes = [ {"name": "fullname", "value": name}, {"name": "skill", "value": "PyDentity SSI Ninja"}, {"name": "age", "value": age} ]

The notebook will ask you to input the identity holder's full name and age which will be used to populate the schema above with the identity holders attribute information.

In [8]:
# from datetime import datetime

name=input("Please enter your name and surname: ")
age=input("Please enter your age: ")
TreatmentDate=input("Please enter treatment date: ")
doctor=input("Please enter doctor's name: ")
ChiefComplaint=input("Please enter chief complaint: ")
examination=input("Please enter examination: ")
treatment=input("Please enter treatment: ")
PrescriptionPad=input("Please enter prescription pad: ")
appointment=input("Please enter appointment: ")

credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "age", "value": age},
    {"name": "date", "value": TreatmentDate},
    {"name": "doctorname", "value": doctor},
    {"name": "complaint", "value": ChiefComplaint},
    {"name": "examination results", "value": examination},
    {"name": "final treatment", "value": treatment},
    {"name": "medicine", "value": PrescriptionPad},
    {"name": "nextappointment", "value": appointment}
]
print(credential_attributes)

Please enter your name and surname:  tim
Please enter your age:  21
Please enter treatment date:  01/01/23
Please enter doctor's name:  amy
Please enter chief complaint:  no
Please enter examination:  no
Please enter treatment:  no
Please enter prescription pad:  no
Please enter appointment:  no


[{'name': 'fullname', 'value': 'tim'}, {'name': 'age', 'value': '21'}, {'name': 'date', 'value': '01/01/23'}, {'name': 'doctorname', 'value': 'amy'}, {'name': 'complaint', 'value': 'no'}, {'name': 'examination results', 'value': 'no'}, {'name': 'final treatment', 'value': 'no'}, {'name': 'medicine', 'value': 'no'}, {'name': 'nextappointment', 'value': 'no'}]


### 5. Go to the_wallet notebook to setup

### 6. Send Credential

his sends a credential to a identity holder (User), and automates the rest of the protocol.

There are other ways to issue a credential that require multiple api calls.

Arguments

- connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
- schema_id: The id of the schema you wish to issue
- cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
- attributes: A list of attribute objects as defined above. Must match the schema attributes.
- comment (optional): Any string, defaults to ""
- auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
- trace (optional): Boolean, defaults to False. Not entirely sure about this one, believe its for logging. Also when set to True it throws an error

In [9]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 0e452b3e-c431-4b2c-8254-e55d25549929, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 0e452b3e-c431-4b2c-8254-e55d25549929, role: issuer, state: offer_sent
Offering: [{'name': 'fullname', 'value': 'tim'}, {'name': 'age', 'value': '21'}, {'name': 'date', 'value': '01/01/23'}, {'name': 'doctorname', 'value': 'amy'}, {'name': 'complaint', 'value': 'no'}, {'name': 'examination results', 'value': 'no'}, {'name': 'final treatment', 'value': 'no'}, {'name': 'medicine', 'value': 'no'}, {'name': 'nextappointment', 'value': 'no'}]
Handle Credentials
Credential exchange 0e452b3e-c431-4b2c-8254-e55d25549929, role: issuer, state: request_received
Offering: [{'name': 'fullname', 'value': 'tim'}, {'name': 'age', 'value': '21'}, {'name': 'date', 'value': '01/01/23'}, {'name': 'doctorname', 'value': 'amy'}, {'name': 'complaint', 'value': 'no'}, {'name': 'examination results', 'value': 'no'}, {'name': 'final treatment', 'value': 'no'}, {'name': 'medicine', 'val

### **Terminate**


In [10]:
response = await agent_controller.terminate()
print(response)

None
